In [12]:
import os
import numpy as np
import pandas as pd
import sklearn as sk
import scipy
from sklearn.metrics import auc, recall_score, precision_score, f1_score, accuracy_score, roc_auc_score, brier_score_loss, plot_roc_curve, confusion_matrix
from sklearn.model_selection import StratifiedKFold, KFold, GridSearchCV, RepeatedStratifiedKFold
from sklearn.feature_selection import SelectFromModel, RFECV, SelectKBest, mutual_info_classif, chi2, f_classif
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer


import matplotlib.pyplot as plt
import seaborn as sns

# import unicodedata # for feature names
import lightgbm

import dill
from datetime import datetime

In [17]:
def dump_model(model, feature_names=None, scores=None, name=None, desc=None, misc=None, path='./'):
#         self.model = model # model pipeline compatible with sklearn API. Contains classifier functions, feature importances etc.
#         self.name = name # used as filename, too
#         self.desc = desc # textual description of model, which data it was trained on, etc.
#         self.scores = scores # Eval scores, sensitivity, specificity, ... as dict
#         self.feature_names = feature_names # feature names get lost in np; should be list(df.columns)
#         self.misc = misc # Dict to store additional information not covered above

#     def dump(self, path='./'):
    if not name:
        name = type(model).__name__ +'_'+ datetime.now().strftime("%Y%m%d-%Hh%Mm%Ss")

    FASDModel = {
        
        "model": model,
        "scores": scores,
        "feature_names": feature_names,
        "name": name,
        "desc": desc,
        "misc": misc
    }

    dill.dump( FASDModel , open( path+name+".p", "wb" ) )
    print("FASDModel dumped as", name)
    
class InputTransformedPipeline():
#     The pipelines expect 14 input features, even for models trained on 6 features, because all
#     features have been used for imputation. 
#     This class allows to hand a dense input vector of the 6 selected features to the pipeline 
    
    def __init__(self, clf):
        self.clf = clf
    
    def predict_proba(self, fs6_array):
        
        nan = float("nan")
        feature_idx =  [0,3,6,8,10,12]
        feats = [nan] * 14
        
        for idx, value in enumerate(fs6_array[0]):
            feats[feature_idx[idx]] = value
        feats = np.array(feats).reshape(1, -1)
        return self.clf.predict_proba(feats)
    
    def predict(self, fs6_array):
        
        nan = float("nan")
        feature_idx =  [0,3,6,8,10,12]
        feats = [nan] * 14
        
        for idx, value in enumerate(fs6_array[0]):
            feats[feature_idx[idx]] = value
        feats = np.array(feats).reshape(1, -1)
        return self.clf.predict(feats)


In [3]:
#########################
# Training on full data #
#########################

# For making an ensemble, use the readily trained models from the Combined ML notebook

clf = RandomForestClassifier()

param_grid_sf_rf = param_grid = { 'imputer__n_neighbors' : [1, 5, 10],
              'imputer__weights' : ['distance'],
              'clf__n_estimators': [10, 20, 50, 100, 200],
              'clf__min_samples_split' : [2, 4, 8, 16],
              'clf__n_jobs' : [-1],
             }

data_path = '../data/adhs-fasd-17032021.xlsx'
missing_values_thresh = 36

# data = pd.read_excel(data_path)
# on jupyterhub setup, we need openpyxl for reading the data 
data = pd.read_excel(data_path, engine='openpyxl')
data = data.select_dtypes(
    include=['float64','int64'],
    exclude=['object']
    )

# features that are very easy to assess
light = [    
    'GKUzscore',
    'GGzscore',
    'KgEVzscore', 
    'GLzscore',
    'KLEVzscore', 
    'Gestationsalter',
    'Schlafstörungen', 
    'Para' 
] 

# features that are elaborate to assess
extra = [
     'IQunter85',    
     'psychKomorbiditätjanein',
     'Distanzlos', 
     'GestörteSprache', 
     'Nichtmerkfähig', 
]

extended = light + extra + ['FASDjanein']

features = extended
X = data[features].to_numpy()
y = data['FASDjanein'].to_numpy()

pipe = Pipeline([('scaler', sk.preprocessing.RobustScaler()),
                 ('imputer', KNNImputer(n_neighbors=2, weights="distance")),
                 ('select_feats', ColumnTransformer([("fs", "passthrough", [0,3,6,8,10,12])], remainder="drop",)),
                 ('clf', clf)])

score_to_optimize = 'roc_auc'
inner_cv_folds = 10

scoring = {'precision_macro': precision_score, 
       'recall_macro': recall_score, 
        'specificity': recall_score,
       'f1_macro': f1_score, 
       'accuracy': accuracy_score, 
       'roc_auc': roc_auc_score, 
        'brier': brier_score_loss
      }

# configure the cross-validation procedure
inner_cv = StratifiedKFold(n_splits=inner_cv_folds, shuffle=True, random_state=1)

# define search    
grid_search = GridSearchCV(pipe, 
                       param_grid=param_grid, 
                       scoring=score_to_optimize, 
                       refit=True, 
                       return_train_score=True, 
                       cv=inner_cv,
                       n_jobs=-1)

# execute search
search_result = grid_search.fit(X, y)

# get the best performing model fit on the whole training set
best_model = search_result.best_estimator_

In [18]:
export_model = InputTransformedPipeline(best_model)
dump_model(export_model, feature_names=['GKUzscore', 'GLzscore', 'Schlafstörungen', 'IQunter85', 'Distanzlos', 'Nichtmerkfähig'], name='FASDetect_RF'+'_'+ datetime.now().strftime("%Y%m%d-%Hh%Mm%Ss"))

FASDModel dumped as FASDetect_RF_20210927-15h10m03s


## Tests

In [15]:
nan = float("nan")
test = np.array([1, nan, nan, 2, nan, nan, 0, nan, 1, nan, 0, nan, 1, nan]).reshape(1, -1)
best_model.predict_proba(test)

array([[0.48670233, 0.51329767]])

In [16]:
type(best_model['clf'])

sklearn.ensemble._forest.RandomForestClassifier

In [100]:
export_model = InputTransformedPipeline(best_model)

In [102]:
nan = float("nan")
test = np.array([1, nan, nan, 2, nan, nan, 0, nan, 1, nan, 0, nan, 1, nan]).reshape(1, -1)
best_model.predict_proba(test)

array([[0.64732874, 0.35267126]])

In [103]:
nan = float("nan")
test = np.array([1, 2, 0, 1, 0, 1]).reshape(1, -1)
export_model.predict_proba(test)

array([[0.64732874, 0.35267126]])

In [38]:
dump_model(search_result.best_estimator_, fs6)

FASDModel dumped as Pipeline_20210601-18h42m57s


In [12]:
search_result.best_score_

0.9115818549642078

In [13]:
search_result.best_params_

{'clf__boosting': 'gbdt',
 'clf__learning_rate': 0.05,
 'clf__max_bin': 127,
 'clf__num_iterations': 50,
 'imputer__n_neighbors': 10,
 'imputer__weights': 'uniform'}

In [14]:
search_result.best_estimator_

Pipeline(steps=[('scaler', RobustScaler()),
                ('imputer', KNNImputer(n_neighbors=10)),
                ('clf',
                 LGBMClassifier(boosting='gbdt', learning_rate=0.05,
                                max_bin=127, num_iterations=50))])

In [34]:
nan = float("nan")
test = np.array([-0.46, -1.41, nan, nan, nan, nan]).reshape(1, -1)
search_result.best_estimator_.predict_proba(test)

array([[0.04825316, 0.95174684]])

In [39]:
nan = float("nan")
test = np.array([nan, nan, nan, nan, nan, nan]).reshape(1, -1)
search_result.best_estimator_.predict_proba(test)

array([[0.03187579, 0.96812421]])

In [41]:
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

pipe_oversampling = Pipeline([('scaler', sk.preprocessing.RobustScaler()),
             ('imputer', KNNImputer(n_neighbors=2, weights="distance")),
             ('oversampling', SMOTE(sampling_strategy=1.0)),
             ('clf', clf)])

inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

grid_search_oversampling = GridSearchCV(pipe_oversampling, 
                       param_grid=param_grid, 
                       scoring='roc_auc', 
                       refit=True, 
                       return_train_score=True, 
                       cv=inner_cv,
                       n_jobs=-1)

# execute search
search_result_oversampling = grid_search.fit(X, y)

/home/jupyter-lukas.ehrig/.local/lib/python3.7/site-packages/lightgbm/engine.py:151: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [44]:
nan = float("nan")
test = np.array([nan, nan, nan, nan, nan, nan]).reshape(1, -1)
search_result_oversampling.best_estimator_.predict_proba(test)

array([[0.03187579, 0.96812421]])

In [10]:
# data_path = '../data/adhs-fasd-17032021.xlsx'
# missing_values_thresh = 36
# mice_iterations = 50

# # data = pd.read_excel(data_path)
# # on jupyterhub setup, we need openpyxl for reading the data 
# data = pd.read_excel(data_path, engine='openpyxl')

# data = data.select_dtypes(
#     include=['float64','int64'],
#     exclude=['object']
#     )

# n_missing_nofas = (data[(data.FASDjanein == 0)].isnull().sum() * 100 / len(data[(data.FASDjanein == 0)]))
# n_missing_nofas_thresh = set(n_missing_nofas[n_missing_nofas < missing_values_thresh].index)

# n_missing_fas = (data[(data.FASDjanein == 1)].isnull().sum() * 100 / len(data[(data.FASDjanein == 1)]))
# n_missing_fas_thresh = set(n_missing_fas[n_missing_fas < missing_values_thresh].index)

# features_thresh = list(n_missing_fas_thresh.intersection(n_missing_nofas_thresh))

# # n_missing_fas_35.union(n_missing_nofas_35).difference(n_missing_fas_35.intersection(n_missing_nofas_35))

# fs6 = [
#     'GKUzscore',
#     'GLzscore',
#     'IQunter85',
#     'Distanzlos',
#     'Nichtmerkfähig',
#     'Schlafstörungen'
# ]

# features = fs6

# X = data[features].to_numpy()
# y = data['FASDjanein'].to_numpy()

# clf = lightgbm.LGBMClassifier()

# param_grid = { 'imputer__n_neighbors' : [1, 4, 6, 10],
#               'imputer__weights' : ['distance', 'uniform'],
#               'clf__max_bin': [127, 255, 511],
#               'clf__learning_rate': [0.05, 0.1, 0.2, 0.5, 1],
#               'clf__num_iterations': [10, 20, 50, 100],
#               'clf__boosting': ['gbdt', 'dart']
#              }

# pipe = Pipeline([('scaler', sk.preprocessing.RobustScaler()),
#                  ('imputer', KNNImputer(n_neighbors=2, weights="distance")),
#                  ('clf', clf)])

# inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

# grid_search = GridSearchCV(pipe, 
#                        param_grid=param_grid, 
#                        scoring='roc_auc', 
#                        refit=True, 
#                        return_train_score=True, 
#                        cv=inner_cv,
#                        n_jobs=-1)

# # execute search
# search_result = grid_search.fit(X, y)

/home/jupyter-lukas.ehrig/.local/lib/python3.7/site-packages/lightgbm/engine.py:151: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
